In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import sys

dir2 = os.path.abspath("")
dir1 = os.path.dirname(dir2)
if dir1 not in sys.path:
    sys.path.append(dir1)

os.chdir("..")

In [13]:
import numpy as np
import torch
import yaml
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import CometLogger
from src.datasets.datasets import CPDDatasets
from src.ensembles.bayes_models import BaseBayesRnn, BayesCPDModel
from src.metrics.evaluation_pipelines import evaluation_pipeline
from src.utils.fix_seeds import fix_seeds
from torch.utils.data import DataLoader

SEED = 42
fix_seeds(SEED)

%load_ext autoreload
%autoreload
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# HAR

In [16]:
model_type = "seq2seq"

experiments_name = "human_activity"
path_to_config = "configs/" + experiments_name + "_" + model_type + ".yaml"

with open(path_to_config, "r") as f:
    args_config = yaml.safe_load(f.read())

args_config["experiments_name"] = experiments_name
args_config["model_type"] = model_type

args_config["loss_type"] = "bce"
args_config["num_workers"] = 2

train_dataset, test_dataset = CPDDatasets(experiments_name).get_dataset_()
test_dataloader = DataLoader(
    test_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=False
)

In [17]:
PRIOR_MU = 0.0
PRIOR_SIGMA = 0.1

KL_COEFF = 0.2

SEED = 3

base_model = BaseBayesRnn(
    input_size=args_config["model"]["input_size"],
    hidden_dim=args_config["model"]["hidden_dim"],
    n_layers=args_config["model"]["n_layers"],
    drop_prob=args_config["model"]["drop_prob"],
    prior_mu=PRIOR_MU,
    prior_sigma=PRIOR_SIGMA,
)

bayes_model = BayesCPDModel(
    args=args_config,
    model=base_model,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    kl_coeff=KL_COEFF,
    n_samples=10,
    seed=SEED,
)

In [ ]:
fix_seeds(SEED)

# model_name = f'{experiments_name}_{args_config["loss_type"]}_model_num_{seed}'
model_name = f"{experiments_name}_bayes_{model_type}_model_num_{SEED}"

logger = CometLogger(
    save_dir=f"logs/{experiments_name}",
    api_key="agnHNC2vEt7tOxnnxT4LzYf7Y",
    project_name="cpd-with-uncertainty",
    workspace="stalex2902",
    experiment_name=model_name,
)

trainer = Trainer(
    max_epochs=args_config["learning"]["epochs"],
    gpus=1,
    benchmark=True,
    check_val_every_n_epoch=1,
    gradient_clip_val=args_config["learning"]["grad_clip"],
    logger=logger,
    callbacks=EarlyStopping(**args_config["early_stopping"]),
)
trainer.fit(bayes_model)

In [19]:
threshold_number = 300
threshold_list = np.linspace(-5, 5, threshold_number)
threshold_list = 1 / (1 + np.exp(-threshold_list))
threshold_list = [-0.001] + list(threshold_list) + [1.001]

# metrics, (max_th_f1_margins_dict, max_f1_margins_dic), _, _
metrics = evaluation_pipeline(
    bayes_model,
    test_dataloader,
    threshold_list,
    device="cpu",
    model_type="ensemble",
    # scale=args_config["predictions"]["scale"],
    verbose=True,
    margin_list=args_config["evaluation"]["margin_list"],
)

  0%|          | 0/21 [00:00<?, ?it/s]

Collectting model's outputs


100%|██████████| 21/21 [00:03<00:00,  6.40it/s]


TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 

In [20]:
path_to_models_folder = "saved_models/total_bayes/human_activity/ens_3"

torch.save(bayes_model.state_dict(), path_to_models_folder + "/" + model_name + ".pth")

# Synthetic 1D

In [26]:
model_type = "seq2seq"

experiments_name = "synthetic_1D"
path_to_config = "configs/" + experiments_name + "_" + model_type + ".yaml"

with open(path_to_config, "r") as f:
    args_config = yaml.safe_load(f.read())

args_config["experiments_name"] = experiments_name
args_config["model_type"] = model_type

args_config["loss_type"] = "bce"
args_config["num_workers"] = 2

train_dataset, test_dataset = CPDDatasets(experiments_name).get_dataset_()
test_dataloader = DataLoader(
    test_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=False
)

In [27]:
PRIOR_MU = 0.0
PRIOR_SIGMA = 0.1

KL_COEFF = 0.2

SEED = 3

base_model = BaseBayesRnn(
    input_size=args_config["model"]["input_size"],
    hidden_dim=args_config["model"]["hidden_dim"],
    n_layers=args_config["model"]["n_layers"],
    drop_prob=args_config["model"]["drop_prob"],
    prior_mu=PRIOR_MU,
    prior_sigma=PRIOR_SIGMA,
)

bayes_model = BayesCPDModel(
    args=args_config,
    model=base_model,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    kl_coeff=KL_COEFF,
    n_samples=10,
    seed=SEED,
)

In [ ]:
fix_seeds(SEED)

# model_name = f'{experiments_name}_{args_config["loss_type"]}_model_num_{seed}'
model_name = f"{experiments_name}_bayes_{model_type}_model_num_{SEED}"

logger = CometLogger(
    save_dir=f"logs/{experiments_name}",
    api_key="agnHNC2vEt7tOxnnxT4LzYf7Y",
    project_name="cpd-with-uncertainty",
    workspace="stalex2902",
    experiment_name=model_name,
)

trainer = Trainer(
    max_epochs=args_config["learning"]["epochs"],
    gpus=1,
    benchmark=True,
    check_val_every_n_epoch=1,
    gradient_clip_val=args_config["learning"]["grad_clip"],
    logger=logger,
    callbacks=EarlyStopping(**args_config["early_stopping"]),
)
trainer.fit(bayes_model)

In [29]:
threshold_number = 300
threshold_list = np.linspace(-5, 5, threshold_number)
threshold_list = 1 / (1 + np.exp(-threshold_list))
threshold_list = [-0.001] + list(threshold_list) + [1.001]

# metrics, (max_th_f1_margins_dict, max_f1_margins_dic), _, _
metrics = evaluation_pipeline(
    bayes_model,
    test_dataloader,
    threshold_list,
    device="cpu",
    model_type="ensemble",
    # scale=args_config["predictions"]["scale"],
    verbose=True,
    margin_list=args_config["evaluation"]["margin_list"],
)

  0%|          | 0/5 [00:00<?, ?it/s]

Collectting model's outputs


100%|██████████| 5/5 [00:03<00:00,  1.44it/s]


TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0

In [30]:
path_to_models_folder = "saved_models/total_bayes/synthetic_1D/ens_3"

torch.save(bayes_model.state_dict(), path_to_models_folder + "/" + model_name + ".pth")